# The C.I.A World Factbook
## Population Information By Country

The World Factbook is maintained by the US Government and is one of its most accessed documents.

"The World Factbook provides basic intelligence on the history, people, government, economy, energy, geography, environment, communications, transportation, military, terrorism, and transnational issues for 266 world entities."

[Click](https://www.cia.gov/the-world-factbook/about/) for more information.

**World Population**

This project explores human distribution throughout the world using factbook data on land area, population and growth, as well as birth, death and migration rates of each country.

**Data Exploration**

SQL queries to the factbook database is used to identify remarkable or superlative characteristics which are subsequently investigated in greater detail in the factbook.

## Summary of Findings

**Low Density**

- The country with the smallest population is Antarctica with 0 permanent population, hosting only temporary residents at scientific research stations.

**High Density** 

- The densest country is the Chinese island nation of Macau with over 20,000 people per sq km.

- The Vatican City is the smallest nation with a permanent population, so small that the area stored in the factbook database is 0. Despite a small population, its miniscule size (0.44 sq km) makes it one of the 10 most dense countries in the world.

**Slow Growth**

- Namibia and Ukraine among the countries of above average area but below average population with the slowest population growth. Both countries rank in the top 10 for highest death rates.  

- Populous countries tend to be more populous due to high population growth, except in the case of Japan whose very low growth rate means that 29% of their population is over 65.

**Fast Growth**

- After India and China, Nigeria is poised to have the highest increase in population and become the 4th most populous country in the world with a whopping 41% of the population below the age of 15.

- Ethiopia and the Democratic Republic of the Congo are both among the top 10 contributors to the global population increase due to a high birth rate from an already moderately-high existing population, and not the result of low migration or death rates.

## Exploring SQLite Factbook Database

- Initiate SQL and load factbook into database.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

- View infomation on tables in database.

In [3]:
%%sql
SELECT * 
  FROM sqlite_master
 WHERE type='table';


Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


- View the first 5 rows of the factbook data.

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


- View min and max of population related data.

In [5]:
%%sql
SELECT MIN(population) 'min pop', 
       MAX(population) 'max pop', 
       MIN(population_growth) 'min pop growth',
       MAX(population_growth) 'max pop growth'
  FROM facts;

Done.


min pop,max pop,min pop growth,max pop growth
0,7256490011,0.0,4.02


- Identify the countries with the most extreme population data.

In [6]:
%%sql
SELECT name, population
  FROM facts
 WHERE population = (SELECT Min(population)
                       FROM facts) 
    OR population = (SELECT Max(population)
                       FROM facts);

Done.


name,population
Antarctica,0
World,7256490011


**Antarctica: population = 0**

Per Factbook:
[Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society) has "no indigenous inhabitants, but there are both permanent and summer-only staffed research station".

- View the superlative population summary statistics, excluding the 'World' row.

In [7]:
%%sql
SELECT name, population
  FROM facts
 WHERE population = (SELECT Min(population)
                       FROM facts) 
    OR population = (SELECT Max(population)
                       FROM facts
                      WHERE name != 'World');

Done.


name,population
China,1367485388
Antarctica,0


- View top 3 most populous countries in the world.

In [8]:
%%sql 
SELECT name, population, population_growth, birth_rate
  FROM facts
 WHERE name != 'World'
 ORDER BY population DESC
 LIMIT 2;
    

Done.


name,population,population_growth,birth_rate
China,1367485388,0.45,12.49
India,1251695584,1.22,19.55


**China: population 1,367,485,388** continues to be the most populous country in the world.
[China](https://www.cia.gov/the-world-factbook/countries/china/#people-and-society)

**India: population(1,251,695,584)** is the 2nd most populous country in the world but with a higher birth rate and rate of population growth, [when] will its population surpass China's ?

- View average population and area across all countries.

In [9]:
%%sql
SELECT CAST(ROUND(AVG(population),0) AS INT) 'avg pop',
       CAST(ROUND(AVG(area),0) AS INT) 'avg area'
  FROM facts;

Done.


avg pop,avg area
62094928,555094


- Identify higher density countries by filtering for above-average population but below-average area (sort by highest to lowest population).

In [10]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts)
   AND area < (SELECT AVG(area)
                 FROM facts)
 ORDER BY population DESC;

Done.


name,population,area
Bangladesh,168957745,148460
Japan,126919659,377915
Philippines,100998376,300000
Vietnam,94348835,331210
Germany,80854408,357022
Thailand,67976405,513120
United Kingdom,64088222,243610


- Generate actual population density for the countries identified above.

In [11]:
%%sql
SELECT name, population/area 'density'
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts)
   AND area < (SELECT AVG(area)
                 FROM facts)
 ORDER BY density DESC;

Done.


name,density
Bangladesh,1138
Philippines,336
Japan,335
Vietnam,284
United Kingdom,263
Germany,226
Thailand,132


- Identify the highest density countries (population/area).

In [12]:
%%sql
SELECT name, population/area 'density', population, area
  FROM facts
  ORDER BY density DESC
  LIMIT 10;

Done.


name,density,population,area
Macau,21168,592731,28
Monaco,15267,30535,2
Singapore,8141,5674472,697
Hong Kong,6445,7141106,1108
Gaza Strip,5191,1869055,360
Gibraltar,4876,29258,6
Bahrain,1771,1346613,760
Maldives,1319,393253,298
Malta,1310,413965,316
Bermuda,1299,70196,54


**Macau: population density 21,168 ppl/sqkm** is the densest country in the world with over half a million people sharing just 28 sqkm.  

[Macau](https://www.cia.gov/the-world-factbook/countries/macau/), a Chinese island, was colonized by the Portuguese in the 16th century and has flourished as a free-market economy for the last 50 years. 
[Hong Kong](https://www.cia.gov/the-world-factbook/countries/hong-kong/#introduction), another Chinese island and the 4th densest country, was colonized by the British in the mid 19th century and has flourished as a free-market economic hub in the region for the last 50 years as well.

- View the population density information for all countries the same size or smaller than the densest country (Macau), sorted by most to least populous.

In [13]:
%%sql
SELECT name, population/area, population, area
  FROM facts
 WHERE area <= (SELECT area 
                  FROM facts
                 WHERE population/area = (SELECT MAX(population/area)
                                            FROM facts))
 ORDER BY population DESC
 LIMIT 10;

Done.


name,population/area,population,area
Macau,21168,592731,28
Monaco,15267,30535,2
Gibraltar,4876,29258,6
Tuvalu,418,10869,26
Nauru,454,9540,21
Tokelau,111,1337,12
Holy See (Vatican City),None,842,0
Cocos (Keeling) Islands,42,596,14
Ashmore and Cartier Islands,None,None,5
Coral Sea Islands,None,None,3


In [14]:
%%sql
SELECT name, 0.44 'actual area', ROUND(population/0.44) 'actual density', area 'original area', population/area as 'density'
 FROM facts
 WHERE name = 'Holy See (Vatican City)';

Done.


name,actual area,actual density,original area,density
Holy See (Vatican City),0.44,1914.0,0,None


**Holy See (Vatican City): ACTUAL density 1914 ppl/sq km** is among the top 10 densest countries in the world.

The [Vatican City](https://www.cia.gov/the-world-factbook/countries/holy-see-vatican-city/)'s land area is stored in the database as 0 so no density could be calculated.  With an actual area of 0.44 sq km, the population density is 1914 ppl/sq km.  

- Identify countries with above-average area and below-average population and view those with the lowest population growth.

In [15]:
%%sql
SELECT name, population, area, ROUND(CAST(population AS FLOAT)/area,2) 'density', population_growth, death_rate
  FROM facts
 WHERE population < (SELECT AVG(population)
                       FROM facts)
   AND area > (SELECT AVG(area)
                 FROM facts)
   AND population_growth < 1
 ORDER BY population_growth;

Done.


name,population,area,density,population_growth,death_rate
Greenland,57733,2166086,0.03,0.0,8.49
Namibia,2212307,824292,2.68,0.59,13.91
Ukraine,44429471,603550,73.61,0.6,14.46
Canada,35099836,9984670,3.52,0.75,8.42
Chile,17508260,756102,23.16,0.82,6.0
Argentina,43431886,2780400,15.62,0.93,7.33
Peru,30444999,1285216,23.69,0.97,6.01


In [16]:
%%sql
SELECT name,  death_rate
  FROM facts
 ORDER BY death_rate DESC
 LIMIT 10;

Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8


**Namibia: population growth 0.59 and Ukraine: population growth 0.6** are among the countries of above-average area but below-average population with the slowest population growth.

[Greenland](https://www.cia.gov/the-world-factbook/countries/greenland/#environment) has a density of only 0.3 and a population growth near to 0. With only 0.6% agricultural land and 80% ice-capped, the low expected growth is not surprising.  

[Namibia](https://www.cia.gov/the-world-factbook/countries/namibia/#people-and-society) and the [Ukraine]() both are also among the top 10 countries with the highest death rate, which may be a contributing factor to their low growth. 

## Grouping Countries by Population and Area

The following section groups the countries by population and area (P/A group)
 
>'population group' values:

> P1: Below 1M

> P2: 1M-10M

> P3: 10M-100M

> P4: Above 100M

>'area group' values: 

> A1: Below 10k sq km

> A2: 10k-1M sq km

> A3: Above 1M sq km

 - View a random subset of new population (P) group and area (A) group values.

In [30]:
%%sql
SELECT name,
  CASE 
    WHEN population < 1000000 THEN 'P1: Below 1M'
    WHEN population <= 10000000 THEN 'P2: 1M-10M'
    WHEN population <= 100000000 THEN 'P3: 10M-100M'
    WHEN population > 100000000 THEN 'P4: Above 100M'
  END AS 'population group',
  CASE
    WHEN area < 10000 THEN 'A1: Below 10k'
    WHEN area <= 1000000 THEN 'A2: 10k-1M'
    WHEN area > 1000000 THEN 'A3: Above 1M'
  END AS 'area group'
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY RANDOM()
 LIMIT 10;

Done.


name,population group,area group
Moldova,P2: 1M-10M,A2: 10k-1M
Israel,P2: 1M-10M,A2: 10k-1M
New Caledonia,P1: Below 1M,A2: 10k-1M
Saint Pierre and Miquelon,P1: Below 1M,A1: Below 10k
Ghana,P3: 10M-100M,A2: 10k-1M
Tanzania,P3: 10M-100M,A2: 10k-1M
Syria,P3: 10M-100M,A2: 10k-1M
Lebanon,P2: 1M-10M,A2: 10k-1M
Aruba,P1: Below 1M,A1: Below 10k
Malaysia,P3: 10M-100M,A2: 10k-1M


- View all new population (P) group and area (A) group values sorted first by P/A combinations and then by population growth.

In [18]:
%%sql
SELECT name,
  CASE 
    WHEN population < 1000000 THEN 'P1: Below 1M'
    WHEN population <= 10000000 THEN 'P2: 1M-10M'
    WHEN population <= 100000000 THEN 'P3: 10M-100M'
    WHEN population > 100000000 THEN 'P4: Above 100M'
  END AS 'population group',
  CASE
    WHEN area < 10000 THEN 'A1: Below 10k'
    WHEN area <= 1000000 THEN 'A2: 10k-1M'
    WHEN area > 1000000 THEN 'A3: Above 1M'
  END AS 'area group',
  population_growth
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY
      CASE 
      WHEN population < 1000000 THEN 'P1: Below 1M'
      WHEN population <= 10000000 THEN 'P2: 1M-10M'
      WHEN population <= 100000000 THEN 'P3: 10M-100M'
      WHEN population > 100000000 THEN 'P4: Above 100M'
      END DESC, 
      CASE
        WHEN area < 10000 THEN 'A1: Below 10k'
        WHEN area <= 1000000 THEN 'A2: 10k-1M'
        WHEN area > 1000000 THEN 'A3: Above 1M'
      END DESC,
      population_growth DESC;
  

Done.


name,population group,area group,population_growth
India,P4: Above 100M,A3: Above 1M,1.22
Mexico,P4: Above 100M,A3: Above 1M,1.18
Indonesia,P4: Above 100M,A3: Above 1M,0.92
United States,P4: Above 100M,A3: Above 1M,0.78
Brazil,P4: Above 100M,A3: Above 1M,0.77
China,P4: Above 100M,A3: Above 1M,0.45
Russia,P4: Above 100M,A3: Above 1M,0.04
Nigeria,P4: Above 100M,A2: 10k-1M,2.45
Philippines,P4: Above 100M,A2: 10k-1M,1.61
Bangladesh,P4: Above 100M,A2: 10k-1M,1.6


- View average values across the P4/A2 group (high population, medium area).

In [19]:
%%sql
SELECT ROUND(AVG(population/area),2) density, ROUND(AVG(population_growth),2) 'growth rate', ROUND(AVG(birth_rate),2) birth_rate, ROUND(AVG(death_rate),2) death_rate, ROUND(AVG(migration_rate),2) migration_rate
  FROM facts
 WHERE population > 100000000
   AND area <= 1000000 
   AND area > 10000;

Done.


density,growth rate,birth_rate,death_rate,migration_rate
451.0,1.46,22.71,8.12,0.86


- Calculate increased of population projected for all the P4/A2 countries using current population and the growth rate.

In [20]:
%%sql
SELECT name, population, population/area density, population_growth as 'growth rate', ROUND(population*population_growth/100,2) 'projected increase of population'
  FROM facts
 WHERE population > 100000000
   AND area <= 1000000 
   AND area > 10000
 ORDER BY ROUND(population*population_growth/100,2) DESC;

Done.


name,population,density,growth rate,projected increase of population
Nigeria,181562056,196,2.45,4448270.37
Pakistan,199085847,250,1.46,2906653.37
Bangladesh,168957745,1138,1.6,2703323.92
Philippines,100998376,336,1.61,1626073.85
Japan,126919659,335,0.16,203071.45


- Rank all countries for highest projected increase of population.

In [21]:
%%sql
SELECT name, 
       ROUND(population*population_growth/100,2) 'projected population increase',
       CASE 
        WHEN population < 1000000 THEN 'P1: Below 1M'
        WHEN population <= 10000000 THEN 'P2: 1M-10M'
        WHEN population <= 100000000 THEN 'P3: 10M-100M'
        WHEN population > 100000000 THEN 'P4: Above 100M'
       END AS 'population group',
       CASE
        WHEN area < 10000 THEN 'A1: Below 10k'
        WHEN area <= 1000000 THEN 'A2: 10k-1M'
        WHEN area > 1000000 THEN 'A3: Above 1M'
       END AS 'area group'
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY ROUND(population*population_growth/100,2) DESC
 LIMIT 10;

Done.


name,projected population increase,population group,area group
India,15270686.12,P4: Above 100M,A3: Above 1M
China,6153684.25,P4: Above 100M,A3: Above 1M
Nigeria,4448270.37,P4: Above 100M,A2: 10k-1M
Pakistan,2906653.37,P4: Above 100M,A2: 10k-1M
Ethiopia,2874562.17,P3: 10M-100M,A3: Above 1M
Bangladesh,2703323.92,P4: Above 100M,A2: 10k-1M
United States,2506677.14,P4: Above 100M,A3: Above 1M
Indonesia,2355141.8,P4: Above 100M,A3: Above 1M
"Congo, Democratic Republic of the",1944690.83,P3: 10M-100M,A3: Above 1M
Philippines,1626073.85,P4: Above 100M,A2: 10k-1M


**Nigeria: projected population increase = 4,448,270** After India and China, Nigeria is poised to have the highest increase in population.

[Nigeria](https://www.cia.gov/the-world-factbook/countries/nigeria/#people-and-society) is on its way to becoming the 4th most populous country in the world. Population growth is fueled by a lack of family planning.  The country is challenged to channel the energy of its young potential workforce into productive activity.

**Japan: population growth 0.16** [Japan](https://www.cia.gov/the-world-factbook/countries/japan/#people-and-society)'s citizens aged 65+make up 29% of the country in contrast with Nigeria's 0-14 year-olds making up a whopping 41% of the population.

- Count how many countries are in the P3/A3 group (med-high population and high area).

In [22]:
%%sql
SELECT COUNT(*) 'num P3/A3 countries'
  FROM facts
WHERE population > 10000000 
  AND population < 100000000
  AND area > 1000000
  AND name != 'European Union';


Done.


num P3/A3 countries
17


- Rank population increase rates (birth rates) in the P3/A3 group.

In [23]:
%%sql
SELECT name, birth_rate, population
  FROM facts
WHERE population > 10000000 
  AND population < 100000000
  AND area > 1000000
  AND name != 'European Union'
 ORDER BY birth_rate DESC
 LIMIT 5;

Done.


name,birth_rate,population
Mali,44.99,16955536
Angola,38.78,19625353
Ethiopia,37.27,99465819
"Congo, Democratic Republic of the",34.88,79375136
Sudan,29.19,36108853


- Rank countries in the P3/A3 group for lowest migration and death rates, view only those countries ranked in the top 10 for both categories. 

In [24]:
%%sql
SELECT name, population_growth 'growth rate',
       birth_rate, death_rate,migration_rate,
       population_growth 'growth rate',
       ROUND(population*population_growth/100,2) 'projected population increase'
  FROM facts
WHERE population > 10000000 
  AND population < 100000000
  AND area > 1000000
  AND migration_rate IN (SELECT migration_rate
                     FROM facts
                        WHERE population > 10000000 
                        AND population < 100000000
                        AND area > 1000000
                     ORDER BY migration_rate
                     LIMIT 10)
 AND death_rate  IN (SELECT death_rate
                     FROM facts
                        WHERE population > 10000000 
                        AND population < 100000000
                        AND area > 1000000
                     ORDER BY death_rate
                     LIMIT 10);

Done.


name,growth rate,birth_rate,death_rate,migration_rate,growth rate_1,projected population increase
Argentina,0.93,16.64,7.33,0.0,0.93,403916.54
Bolivia,1.56,22.76,6.52,0.62,1.56,168493.76
Colombia,1.04,16.47,5.4,0.64,1.04,486061.97
Egypt,1.79,22.9,4.77,0.19,1.79,1583924.39
Iran,1.2,17.99,5.94,0.07,1.2,981891.24
Saudi Arabia,1.46,18.51,3.33,0.55,1.46,405183.81


**[Ethiopia](https://www.cia.gov/the-world-factbook/countries/ethiopia/) and [The DRC](https://www.cia.gov/the-world-factbook/countries/congo-democratic-republic-of-the/)** both countries are among the top 10 contributors to the global population increase due to a high birth rate from an already moderately-high existing population, and not the result of low migration or death rates.

## Future Analysis

For future analysis, it would be nice to tweak the P/A grouping ranges and to perform more statistics comparing countries within and across these groups.  

Other groupings that could yield interesting comparative results would be area_water/area_land proportion.  